The goal of collecting this dataset:
 The dataset on car accidents is collected to enable predictive modeling of traffic incidents. The primary
  objective is to identify patterns and key factors contributing to car accidents, such as collision 
  severity, weather conditions, road types, and contributing elements. By using this data, predictive 
  models can be developed to forecast the likelihood and severity of future accidents, with the aim of 
  improving road safety, optimizing traffic management, and informing policy decisions.

In [ ]:
import pandas as pd

data = pd.read_csv(r"\Users\hoora\OneDrive\Desktop\IT326\monroe county car crach 2003-2015.csv", encoding="windows-1252")  
print(data.info()) 


Link/Resource of this dataset: https://www.kaggle.com/datasets/jacksondivakarr/car-crash-dataset?resource=download - Car Accidents in USA more specific LA-monroe
